In [15]:
!pip install pyspark

In [16]:
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip

In [17]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
      .config('spark.ui.port', '4050')
      .appName("SparkUI Introdução")
      .getOrCreate()
)

In [35]:
!./ngrok authtoken 2cxuq4PjVqz1SCLdCHLetk4KSO3_7YCGS1oZBxK5Hiy83mCh3
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [36]:
from datetime import timedelta
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, TimestampType, DoubleType


schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType()),
])


eschema_pix = StructType([
    StructField('chave_pix', StringType()),
    StructField('fraude', IntegerType()),
    StructField('id_transacao', IntegerType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('transaction_date', TimestampType()),
    StructField('valor', DoubleType())
])



caminho = '/content/pix_transactions.json'

df = spark.read.json(
    caminho,
    schema= eschema_pix,
    timestampFormat = 'yyyy-MM-dd'
    )

In [37]:
df.printSchema()

root
 |-- chave_pix: string (nullable = true)
 |-- fraude: integer (nullable = true)
 |-- id_transacao: integer (nullable = true)
 |-- remetente: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- destinatario: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- transaction_date: timestamp (nullable = true)
 |-- valor: double (nullable = true)



In [38]:
df.show()

+---------+------+------------+--------------------+--------------------+-------------------+--------+
|chave_pix|fraude|id_transacao|           remetente|        destinatario|   transaction_date|   valor|
+---------+------+------------+--------------------+--------------------+-------------------+--------+
|      cpf|     0|        1000|{Jonathan Gonsalv...|{Gabriel Cunha, I...|2022-03-19 00:00:00|    7.05|
|aleatoria|     0|        1001|{Jonathan Gonsalv...|{Diego Souza, XP,...|2021-01-26 00:00:00|   37.28|
|aleatoria|     0|        1002|{Jonathan Gonsalv...|{Nicole Nunes, BT...|2022-05-31 00:00:00|  282.73|
|aleatoria|     0|        1003|{Jonathan Gonsalv...|{Maria Fernanda C...|2022-07-04 00:00:00| 8447.92|
|aleatoria|     0|        1004|{Jonathan Gonsalv...|{Isabel Silva, C6...|2021-09-11 00:00:00|   58.51|
|  celular|     0|        1005|{Jonathan Gonsalv...|{Anthony Carvalho...|2022-02-11 00:00:00| 6655.12|
|      cpf|     0|        1006|{Jonathan Gonsalv...|{Eloah Monteiro, ...|

#Select

In [39]:
df.select('remetente.nome').show()

+------------------+
|              nome|
+------------------+
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
|Jonathan Gonsalves|
+------------------+
only showing top 20 rows



In [41]:
df.select('destinatario.nome').show()

+--------------------+
|                nome|
+--------------------+
|       Gabriel Cunha|
|         Diego Souza|
|        Nicole Nunes|
|Maria Fernanda Ca...|
|        Isabel Silva|
|    Anthony Carvalho|
|      Eloah Monteiro|
|        Sophie Rocha|
|      Pietro Ribeiro|
|      Eloah Teixeira|
|     Emanuella Sales|
|    Valentina Campos|
|       Stella Araujo|
|     Benicio Costela|
|      Joao Fernandes|
|   Gabriela da Rocha|
|      Larissa Aragao|
|           Theo Dias|
|        Danilo Jesus|
|       Bruno Correia|
+--------------------+
only showing top 20 rows

